# Isotropic, isothermal linear elasticity UMAT Description

## Overview

UMAT for isotropic elasticity that incorporates, axisymmetric, plane and 3D stress formulations.

Isotropic elasticity formulation derived from Chapter 4, Introduction to Linear Elasticity, Gould, P. Axisymmetric and plane stress derived from EN2340 Computational Methods in Solid Mechanics, Allan Bower, Brown University. 

## Elastic Properties and Calculated Parameters

In this case the material is defined by its given Young's Modulus, $E$ and Poisson's ratio, $\nu$. The Young's modulus measures the tensile stiffness of a solid, this is achieved by quantifying the relationship between tensile stress $\sigma$ and axial strain $\varepsilon$ in the linear elastic region of a material and is given by the following formula;

\begin{equation}
E = \frac{\sigma}{\varepsilon}
\end{equation}

Poisson's ratio is a measure of material deformation in directions perpendicular to the direction of loading, given by the negative of the ratio of transverse strains, $\varepsilon_{trans}$ to axal strains, $\varepsilon_{axial}$  as shown in the following formula; 

\begin{equation}
    \nu = - \frac{\varepsilon_{trans}}{\varepsilon_{axial}}
\end{equation}

From these properties, Lamé's first parameter, $\mu$ and Lamé's second parameter, $\lambda$ are now defined; 

\begin{equation}
    \mu = \frac{E}{2+(1+\nu)}
\end{equation}

\begin{equation}
    \lambda = \frac{\nu E}{(1+\nu) (1-2 \nu)}
\end{equation}

In addition to these parameters for the plane stress case we will define the following parameter $\kappa$; 

\begin{equation}
    \kappa = \frac{E}{1-\nu^2}
\end{equation}

## 3D Case

For homogeneous and isotropic materials Lamé's parameters define Hooke's law in three dimensions, where the stress tensor in index notation $\sigma_{ij}$ may be defined as;

\begin{equation}
    \sigma_{ij} = 2 \mu \varepsilon_{ij} + \lambda \delta_{ij} \varepsilon_{kk} 
\end{equation}

Where $\varepsilon_{ij}$ is the strain tensor in index notation and $\delta_{ij}$ is the the Kronecker delta function, this is simply a two variable function where the function is equal to one where the variables are equal and zero otherwise, this is expressed below; 

\begin{equation}
    \delta_{ij} = \begin{cases} 0 & \text{if} & i \neq j \\
    1 & \text{if} & i = j 
    \end{cases} 
\end{equation}

In order to implement a UMAT, we recall that the Jacobian or stiffness tensor $C$ must be defined, for linear elasticity because of small deformations the Jacobian is defined as;

\begin{equation}
    C = \frac{\partial \Delta \sigma}{\partial \Delta \varepsilon}
\end{equation}

Where $\Delta \sigma$ is the increment in stress and $\Delta \varepsilon$ is the increment in strain. This stiffness tensor $C$ is a fourth order tensor that forms part of the generalized form of Hooke's Law as seen below expressed in index notation;

\begin{equation}
    \sigma_{ij} = C_{ijkl} \varepsilon_{kl}
\end{equation}

For isotropic linear elastic materials the stiffness tensor in index notation $C_{ijkl}$ can be expressed in terms of Lamé's parameters as;

\begin{equation}
    C_{ijkl} = \lambda \delta_{ij} \delta_{kl} + \mu ( \delta_{ik} \delta_{jl} + \delta_{il} \delta_{jk}) 
\end{equation}

Where the stiffness tensor is written in the following form, shown in tensor notation, denoted by subscripts $ijkl$ and Voigt notation denoted by subscripts $\alpha \beta$ respectively;  

\begin{equation}
    \{C_{ijkl}\} =
    \begin{bmatrix}
        C_{1111} & C_{1122} & C_{1133} & C_{1123} & C_{1113} & C_{1112} \\
        C_{1122} & C_{2222} & C_{2233} & C_{2223} & C_{2213} & C_{2212} \\
        C_{1133} & C_{2233} & C_{3333} & C_{3323} & C_{3313} & C_{3312} \\
        C_{1123} & C_{2223} & C_{3323} & C_{2323} & C_{2313} & C_{2312} \\
        C_{1113} & C_{2213} & C_{3313} & C_{2313} & C_{2323} & C_{2313} \\
        C_{1112} & C_{2212} & C_{3312} & C_{2312} & C_{1312} & C_{1212} 
    \end{bmatrix}
\end{equation}

\begin{equation}
    \{C_{\alpha \beta}\} =
    \begin{bmatrix}
        C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
        C_{12} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
        C_{13} & C_{23} & C_{33} & C_{34} & C_{35} & C_{36} \\
        C_{14} & C_{24} & C_{34} & C_{44} & C_{45} & C_{46} \\
        C_{15} & C_{25} & C_{35} & C_{45} & C_{55} & C_{56} \\
        C_{16} & C_{26} & C_{36} & C_{46} & C_{56} & C_{66} \\
    \end{bmatrix}
\end{equation}

Following the index notation expression of the stiffness tensor, it can then be written in the matrix formulation below;

\begin{equation}
    \begin{bmatrix}
        \sigma_{11} \\
        \sigma_{22} \\
        \sigma_{33} \\
        \sigma_{12} \\
        \sigma_{23} \\
        \sigma_{31}
    \end{bmatrix}
    = 
    \begin{bmatrix}
        2 \mu + \lambda & \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & 2 \mu + \lambda & \lambda & 0 & 0 & 0 \\
        \lambda & \lambda & 2 \mu + \lambda & 0 & 0 & 0 \\
        0 & 0 & 0 & 2 \mu & 0 & 0 \\
        0 & 0 & 0 & 0 & 2 \mu & 0 \\
        0 & 0 & 0 & 0 & 0 & 2 \mu
    \end{bmatrix}
    \begin{bmatrix}
        \varepsilon_{11} \\
        \varepsilon_{22} \\
        \varepsilon_{33} \\
        \varepsilon_{12} \\
        \varepsilon_{23} \\
        \varepsilon_{31}
    \end{bmatrix}
\end{equation}

This stress strain relationship is implemented in the UMAT using the following block of code;


```fortran
DO K1 = 1,NDI
	DO K2 = 1,NDI
		DDSDDE(K2,K1) = ELAMBDA
	END DO
	DDSDDE(K1,K1) = TWO*EMU+ELAMBDA
END DO
	DO K1 = NDI+1,NTENS
        DDSDDE(K1,K1) = TWO*EMU
	END DO
```